## Imports

In [14]:
import tensorflow as tf
from keras import datasets, layers, models
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import os
from time import sleep
import cv2
import pandas as pd

## Making the dataset

In [3]:
# Capture video from webcam and make dataset
class MakeDataset():
    def __init__(self):
        import cv2
        import os
        
        self.video = cv2.VideoCapture(0)
        self.count = 0
        self.person_name = input("Enter person name: ")
        # self.num_images = int(input("Enter number of images: "))
        self.path = "data/" + self.person_name
        if not os.path.exists(self.path):
            os.makedirs(self.path)
    def  make(self):
        while True:
            _, frame = self.video.read()
            cv2.imshow("frame", frame)
            if cv2.waitKey(1) & 0xFF == ord('s'):
                self.count += 1
                cv2.imwrite(self.path + "/" + self.person_name + '_' + str(self.count) + ".jpg", frame)
                print(f'Image saved: {self.count}', end='\r')
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            if self.count == 50:
                break
    def __final__(self):
        self.video.release()
        cv2.destroyAllWindows()

# MakeDataset().make()

## Loading the data

### 1. Listing images in the dataset directory

In [4]:
# # Devasheesh's faces
# deva_faces = './data/devasheesh/'
# deva_faces = os.listdir(deva_faces)
# deva_faces = [deva_faces[i] for i in range(len(deva_faces)) if deva_faces[i].endswith('.jpg')]

# # Swarnim's faces
# swar_faces = './data/swarnim/'
# swar_faces = os.listdir(swar_faces)
# swar_faces = [swar_faces[i] for i in range(len(swar_faces)) if swar_faces[i].endswith('.jpg')]

# # Negative faces
# neg_faces = './data/negative-faces/'
# neg_faces = os.listdir(neg_faces)
# neg_faces = [neg_faces[i] for i in range(len(neg_faces)) if neg_faces[i].endswith('.jpg')]

# Making a function to list all the images in a folder
def list_images(path):
    image_files = []
    for root, _, files in os.walk(path):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_files.append(os.path.join(root, file))
    image_files.sort()
    return image_files

deva_faces = list_images('./data/Devasheesh/')
swar_faces = list_images('./data/Swarnim/')
neg_faces = list_images('./data/negative-faces/')

### 2. Making Train and Test Objects

In [5]:
# function to extract the face from the image
# using the Haar Cascade Classifier
def extract_frontal_face_harr(image_ndarray, grayscale=True, size=(150, 150)):
    # Load the cascade
    face_cascade = cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')
    # Convert into grayscale
    if grayscale:
        image_gray = cv2.cvtColor(image_ndarray, cv2.COLOR_BGR2GRAY)
    else:
        image_gray = image_ndarray
    # Detect faces
    faces_cord = face_cascade.detectMultiScale(image_gray, 1.3, 5)
    # Return the face or None if not found
    if len(faces_cord) == 0:
        return None, None
    # Extract the face
    (x, y, w, h) = faces_cord[0]
    # Resize the image to 150x150
    image_gray_resized = cv2.resize(image_gray[y:y+w, x:x+h], size)
    # Return only the face part of the image
    return image_gray_resized, faces_cord

# using DNN
modelFile = "dnn/res10_300x300_ssd_iter_140000.caffemodel"
configFile = "dnn/deploy.prototxt.txt"
net = cv2.dnn.readNetFromCaffe(configFile, modelFile)

def extract_frontal_face(image_ndarray, grayscale=False, size=(150, 150)):
    # resize the image to 300x300 for the DNN model
    h, w = 300, 300
    image_ndarray = cv2.resize(image_ndarray, (h, w))
    # Convert into blob
    blob = cv2.dnn.blobFromImage(image_ndarray, 1.0, (h, w), (104.0, 177.0, 123.0))
    # Convert into grayscale
    if grayscale:
        image_ndarray = cv2.cvtColor(image_ndarray, cv2.COLOR_BGR2GRAY)
    # Detect faces
    net.setInput(blob)
    detections = net.forward()

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            # Get the coordinates of the bounding box
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (x, y, x2, y2) = box.astype("int")

            # Extract the face ROI (region of interest) from the image
            face = image_ndarray[y:y2, x:x2]
            return cv2.resize(face, size), confidence
        else:
            return None, None


In [7]:
# function to make the image size uniform and make train and test object with labels
class dataframe():
    def __init__(self):
        self.deva_faces = deva_faces
        self.swar_faces = swar_faces
        self.neg_faces = neg_faces
        self.images, self.labels = list(), list()

        self.total_images = len(deva_faces)+len(swar_faces)+len(neg_faces)
        self.labels_list = ['Unknown', 'Devasheesh', 'Swarnim']

        print('Total images: ', self.total_images)

        self.datagen = ImageDataGenerator(
            rotation_range=40,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.2, # randomly shift images vertically (fraction of total height)
            shear_range=0.2,    # set range for random shear
            zoom_range=0.2,    # set range for random zoom
            horizontal_flip=True,   # randomly flip images
            fill_mode='nearest' # set mode for filling points outside the input boundaries
        )

    def append_dataframe(self, images_list, label, augment=False, grayscale=False):
        count = 1
        for image_adr in images_list:
            # read the image
            image = cv2.imread(image_adr)
            # extract the face
            face, _ = extract_frontal_face(image, grayscale=grayscale, size=(150, 150))

            print('Count: ', count, ' | Image address: ', image_adr, ' | Label: ',self.labels_list[label], ' | Image shape: ', image.shape)
            if face is not None:
                # print('Face type: ', type(face))
                # print('Image shape: ', image.shape)
                # print('Face shape: ', face.shape)
                # print('Face: ', plt.imshow(face))

                # append the face image to the list
                if augment:
                    # using the image data generator to augment the images
                    i = 0
                    for batch in self.datagen.flow(face.reshape(1, 150, 150, 3), batch_size=1):
                        # append the image to the list
                        self.images.append(batch[0])
                        self.labels.append(label)
                        i += 1
                        if i > 10:
                            break
                    count += 1

                else:
                    self.images.append(face)
                    self.labels.append(label)
                    count += 1
            else:
                print(f'Face not found in {image_adr}')
                continue
    
    def make_dataframe(self):
        self.append_dataframe(self.neg_faces, 0, augment=False)
        self.append_dataframe(self.deva_faces, 1, augment=True)
        self.append_dataframe(self.swar_faces, 2, augment=True)
        self.images = np.array(self.images)
        self.labels = np.array(self.labels)
        print('Images shape: ', self.images.shape)
        print('Labels shape: ', self.labels.shape)
        return self.images/255.0, self.labels, self.labels_list

x = dataframe()
images, labels, labels_list = x.make_dataframe()

Total images:  175
Count:  1  | Image address:  ./data/negative-faces/n000001/0001_01.jpg  | Label:  Unknown  | Image shape:  (227, 199, 3)
Count:  2  | Image address:  ./data/negative-faces/n000001/0002_01.jpg  | Label:  Unknown  | Image shape:  (511, 477, 3)
Count:  3  | Image address:  ./data/negative-faces/n000001/0003_01.jpg  | Label:  Unknown  | Image shape:  (190, 191, 3)
Count:  4  | Image address:  ./data/negative-faces/n000001/0004_01.jpg  | Label:  Unknown  | Image shape:  (600, 674, 3)
Count:  5  | Image address:  ./data/negative-faces/n000001/0005_01.jpg  | Label:  Unknown  | Image shape:  (199, 174, 3)
Count:  6  | Image address:  ./data/negative-faces/n000001/0006_01.jpg  | Label:  Unknown  | Image shape:  (878, 878, 3)
Count:  7  | Image address:  ./data/negative-faces/n000001/0007_01.jpg  | Label:  Unknown  | Image shape:  (532, 549, 3)
Count:  8  | Image address:  ./data/negative-faces/n000001/0008_01.jpg  | Label:  Unknown  | Image shape:  (295, 310, 3)
Count:  9  | 

In [ ]:
for x in range(len(images)):
    try:
        plt.imshow(images[x])
        plt.title(labels_list[int(labels[x])])
        plt.show()
    except:
        print(f'Error in {x}')

## Splitting the data into train and test

In [8]:
# split the data into train and test
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2)

In [9]:
images.shape

(1545, 150, 150, 3)

In [10]:
train_images.shape

(1236, 150, 150, 3)

In [11]:
test_images.shape

(309, 150, 150, 3)

## Creating The CNN Model

In [17]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150,150,3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(len(labels_list), activation='softmax'))

model.summary()



Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 74, 74, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 34, 34, 64)        36928     
                                                                 
 flatten (Flatten)           (None, 73984)            

In [18]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

2023-02-19 04:01:41.001458: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 333720000 exceeds 10% of free system memory.
2023-02-19 04:01:41.512114: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 333720000 exceeds 10% of free system memory.


Epoch 1/10


/home/swarnim/.local/lib/python3.10/site-packages/keras/backend.py:5585: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
2023-02-19 04:01:44.492984: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8700
2023-02-19 04:01:46.582358: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-19 04:01:46.919607: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x563f9632d320 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-02-19 04:01:46.919667: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce GTX 1050, Compute Capability 6.1
2023-02-19 04:01:46.975373: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.c

39/39 [==============================] - 11s 76ms/step - loss: 0.5547 - accuracy: 0.8293 - val_loss: 0.0858 - val_accuracy: 0.9741
Epoch 2/10
39/39 [==============================] - 2s 51ms/step - loss: 0.0707 - accuracy: 0.9879 - val_loss: 0.0084 - val_accuracy: 1.0000
Epoch 3/10
39/39 [==============================] - 2s 50ms/step - loss: 0.0194 - accuracy: 0.9935 - val_loss: 0.0017 - val_accuracy: 1.0000
Epoch 4/10
39/39 [==============================] - 2s 50ms/step - loss: 6.6522e-04 - accuracy: 1.0000 - val_loss: 6.9355e-05 - val_accuracy: 1.0000
Epoch 5/10
39/39 [==============================] - 2s 50ms/step - loss: 1.1805e-04 - accuracy: 1.0000 - val_loss: 5.0902e-05 - val_accuracy: 1.0000
Epoch 6/10
39/39 [==============================] - 2s 51ms/step - loss: 4.7071e-05 - accuracy: 1.0000 - val_loss: 1.6847e-05 - val_accuracy: 1.0000
Epoch 7/10
39/39 [==============================] - 2s 50ms/step - loss: 2.8273e-05 - accuracy: 1.0000 - val_loss: 1.2995e-05 - val_accuracy

In [19]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print(test_acc)


10/10 - 0s - loss: 7.3699e-06 - accuracy: 1.0000 - 252ms/epoch - 25ms/step
1.0


In [20]:
model.save('face_recognition_model.h5')

In [ ]:
# create a function to predict using webcam
def predict(model, labels_list, grayscale=False):
    # initialize the video capture object
    cap = cv2.VideoCapture(0)
    while True:
        # read the frame from the webcam
        _, frame = cap.read()
        # extract the face
        face, confidence = extract_frontal_face(frame, grayscale=grayscale, size=(150, 150))
        if face is not None:
            # predict the label
            label = np.argmax(model.predict(face.reshape(1, 150, 150, 3)))
            # draw the rectangle
            cv2.rectangle(frame, (x, y), (x2, y2), (0, 255, 0), 2)
            # draw the text
            cv2.putText(frame, f'{labels_list[label]} {confidence*100:.2f}%', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        cv2.imshow('Face Recognition', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()